In [20]:
import numpy as np
import csv

#LOAD DATA CSV menjadi array of dataset dan array of label
def load_csv(filename):
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        line_count = 0
        #MAKE DUMMY ARRAY
        X = np.zeros((1,13))
        label = np.array(None)
        for row in csv_reader:
            if line_count == 0:
                #NOT USING THE COLUMN NAMES
#                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                #ADD ELEMENT AND LABEL TO ARRAY
                line = np.empty(0)
                for i in range(13):
                    if row[i] == "?":
                        line = np.append(line, None)
                    else:
                        line = np.append(line, row[i])
                X = np.append(X, np.array([line]), axis=0)
                label = np.append(label, row[13])
                line_count += 1
        #REMOVE DUMMY ELEMENT
        X = np.delete(X, 0, 0)
        label = np.delete(label, 0)
    return X, label

def sort_X(X):
    arr = [[] for i in range(len(X[0]))]
    for i in range(len(X)):
        for j in range(len(X[i])):
            if X[i][j] != None:
                arr[j].append(X[i][j])
    for i in range(len(X[0])):
        arr[i].sort()
    return arr

def exchange_question_mark(X, arr):
    for i in range(len(X)):
        for j in range(len(X[i])):
            if X[i][j] == None:
                X[i][j] = arr[j][len(arr[j])//2]
    return X

def normalize(X, arr):
    for i in range(len(X)):
        for j in range(len(X[i])):
            #Age
            if j == 0:
                X[i][j] = float(X[i][j]) / float(arr[j][len(arr[j])//2])
            #Cp
            elif j == 2:
                X[i][j] = float(X[i][j]) / float(arr[j][len(arr[j])//2])
            #Trestbps
            elif j == 3:
                X[i][j] = float(X[i][j]) / float(arr[j][len(arr[j])//2])
            #Chol
            elif j == 4:
                X[i][j] = float(X[i][j]) / float(arr[j][len(arr[j])//2])
            #Thalach
            elif j == 7:
                X[i][j] = float(X[i][j]) / float(arr[j][len(arr[j])//2])
            #Thal
            elif j == 12:
                X[i][j] = float(X[i][j]) / float(arr[j][len(arr[j])//2])
    return X

X1, label1 = load_csv('train.csv')
arr1 = sort_X(X1)
X1 = exchange_question_mark(X1, arr1)
X1 = normalize(X1, arr1)

[[1.0408163265306123 '0' 1.0 ... '2' '0' '6']
 [1.1020408163265305 '1' 1.3333333333333333 ... '2' '0' '6']
 [0.9795918367346939 '1' 0.6666666666666666 ... '2' '0' '6']
 ...
 [0.6326530612244898 '1' 1.3333333333333333 ... '2' '0' '6']
 [0.9591836734693877 '1' 0.6666666666666666 ... '2' '0' '6']
 [1.0204081632653061 '1' 1.3333333333333333 ... '2' '0' '6']]


In [16]:
X2, label2 = load_csv('train.csv')
X3, label3 = load_csv('test.csv')
X2 = np.append(X2, X3, axis=0)
label2 = np.append(label2, label3)
arr2 = sort_X(X2)
X2 = exchange_question_mark(X2, arr2)
X2 = normalize(X2, arr2)

In [17]:
from sklearn.cluster import AgglomerativeClustering
clustering_model_1 = AgglomerativeClustering(linkage='complete').fit(X1)
clustering_model_2 = AgglomerativeClustering(linkage='complete').fit(X2)
print(clustering_model_1.labels_)
print(clustering_model_2.labels_)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [18]:
#ASUMSI labelclust dan label ukuran sama
def purity(labelclust, label):
    labelmap = {}
    summax = 0
    for i in range(len(labelclust)):
        if labelclust[i] not in labelmap.keys():
            labelmap[labelclust[i]] = {}
        if label[i] not in labelmap[labelclust[i]].keys():
            labelmap[labelclust[i]][label[i]] = 1
        else:
            labelmap[labelclust[i]][label[i]] += 1
    print(labelmap)
    for el in labelmap.keys():
        summax += max(labelmap[el].values())
    return summax/len(labelclust)
                
print(purity(clustering_model_1.labels_, label1))
print(purity(clustering_model_2.labels_, label2))

{0: {'0': 998, '1': 200}, 1: {'0': 2}}
0.8333333333333334
{0: {'0': 1052, '1': 150}, 1: {'1': 78, '0': 9}}
0.8766485647788984


In [19]:
from sklearn.cluster import DBSCAN
clustering_DBSCAN_model_1 = DBSCAN().fit(X1)
clustering_DBSCAN_model_2 = DBSCAN().fit(X2)
print(purity(clustering_DBSCAN_model_1.labels_, label1))
print(purity(clustering_DBSCAN_model_2.labels_, label2))

{0: {'0': 14, '1': 3}, 1: {'1': 25, '0': 475}, 2: {'0': 9}, -1: {'1': 50, '0': 34}, 3: {'1': 38}, 4: {'0': 14}, 5: {'0': 8}, 6: {'0': 72}, 7: {'0': 120}, 8: {'0': 5}, 9: {'1': 4, '0': 42}, 10: {'0': 23, '1': 5}, 11: {'1': 9, '0': 17}, 12: {'1': 9}, 13: {'0': 23}, 14: {'1': 4, '0': 49}, 15: {'0': 31, '1': 7}, 16: {'1': 39, '0': 19}, 17: {'0': 9}, 18: {'0': 8, '1': 1}, 19: {'1': 6}, 20: {'0': 5}, 21: {'0': 14}, 22: {'0': 9}}
0.9075
{0: {'0': 14, '1': 4}, 1: {'1': 30, '0': 494}, 2: {'0': 9}, -1: {'1': 61, '0': 48}, 3: {'1': 39}, 4: {'0': 14}, 5: {'0': 8}, 6: {'0': 76}, 7: {'0': 137}, 8: {'0': 5}, 9: {'1': 5, '0': 42}, 10: {'0': 23, '1': 5}, 11: {'1': 9, '0': 18}, 12: {'1': 9}, 13: {'0': 25, '1': 1}, 14: {'1': 4, '0': 52}, 15: {'0': 32, '1': 7}, 16: {'1': 42, '0': 19}, 17: {'0': 9}, 18: {'0': 8, '1': 1}, 19: {'1': 6}, 20: {'0': 5}, 21: {'0': 14}, 22: {'0': 9}, 23: {'1': 5}}
0.8968192397207138
